# Some functions performing useful data preprocessing

In [1]:
import h5py
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import scipy
from scipy.spatial.distance import cdist

### Transform data from all cameras into a dataframe containing all features

In [93]:
hdf5data = h5py.File('../../Data/data_109.h5', 'r')

In [23]:
# np.asarray(hdf5data['Sequence'][str(0)]['Detections'])[0]

((-11.14840948, 15.36387945, -0.94192017), 1.17728508, 0.56196702, 1.33484852, 0.6319266)

In [30]:
# a = np.array([1,2,3,4,5])
# a[np.argmin(np.array([[1,2,3], [3,2, -1], [3,2,-1]]), axis=1)]

array([1, 3, 3])

In [49]:
def link_detections_tracks(detections, tracks):
    
    coord_detections = [np.array(vals[0].tolist()) for vals in detections]
    coord_tracks = [np.array(vals[1].tolist()) for vals in tracks]
    vel_tracks = [np.array(vals[2].tolist()) for vals in tracks]
    track_id = [vals[0].tolist() for vals in tracks]
    object_type = [vals[3] for vals in tracks]
    status_tracks = [vals[4] for vals in tracks]
    
    # for the time being, let us associate the closest bounding box, even if it is assigned more than once
    
    dists = cdist(coord_tracks, coord_detections, 'euclidean')
#     print(dists)
    associations = np.argmin(dists, axis=1)
    
    associations = associations.tolist()
    associated_detections = [detections[i] for i in associations]
    coord_detections = [coord_detections[i] for i in associations]
    length_box = [vals[1] for vals in associated_detections]
    width_box = [vals[2] for vals in associated_detections]
    height_box = [vals[3] for vals in associated_detections]
    angle_box = [vals[4] for vals in associated_detections]
    
    coord_detections = np.vstack(coord_detections)
    coord_tracks = np.vstack(coord_tracks)
    vel_tracks = np.vstack(vel_tracks)
    
    returndf = pd.DataFrame()
#     print(coord_tracks)
    returndf["X_track"] = coord_tracks[:,0]
    returndf["Y_track"] = coord_tracks[:,1]
    returndf["Z_track"] = coord_tracks[:,2]
#     print(returndf)
#     print(track_id)
    returndf["vx_track"] = vel_tracks[:,0]
    returndf["vy_track"] = vel_tracks[:,1]
    returndf["vz_track"] = vel_tracks[:,2]
    returndf["track_id"] = track_id
    returndf["obj_type"] = object_type
    returndf["track_status"] = status_tracks
    
    returndf["X_box"] = coord_detections[:,0]
    returndf["Y_box"] = coord_detections[:,1]
    returndf["Z_box"] = coord_detections[:,2]
    returndf["length_box"] = length_box
    returndf["width_box"] = width_box
    returndf["height_box"] = height_box
    returndf["angle_box"] = angle_box
    
    return returndf

In [94]:
def h5_to_df(h5data):
    
    timestamps = h5data['Timestamp']
    df = pd.DataFrame()
    
    for c, t in enumerate(timestamps):
        d = h5data['Sequence'][str(c)]
        track = np.asarray(d['Tracks'])
        detection = np.asarray(d['Detections'])
        if(track.shape[0]!=0 and detection.shape[0]!=0):
            row = link_detections_tracks(detection, track)
            append_df = link_detections_tracks(detection, track)
            append_df['timestamp'] = t
            df = df.append(append_df)
        
    return df

In [95]:
df = h5_to_df(hdf5data)

In [96]:
df.reset_index(inplace=True, drop=True)
df
df['camera'] = '109'

In [72]:
df['obj_type'].value_counts()

18    1252
4     1206
0      495
34     299
1      287
Name: obj_type, dtype: int64

In [97]:
hdf5data = h5py.File('../../Data/data_130.h5', 'r')
newdf = h5_to_df(hdf5data)
newdf['camera'] = '130'
df = df.append(newdf)

hdf5data = h5py.File('../../Data/data_142.h5', 'r')
newdf = h5_to_df(hdf5data)
newdf['camera'] = '142'
df = df.append(newdf)

hdf5data = h5py.File('../../Data/data_143.h5', 'r')
newdf = h5_to_df(hdf5data)
newdf['camera'] = '143'
df = df.append(newdf)

df.reset_index(inplace=True, drop=True)



In [98]:
df

,X_track,Y_track,Z_track,vx_track,vy_track,vz_track,track_id,obj_type,track_status,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,camera
0,6.548051,26.367316,-0.847761,0.207261,0.359813,-0.045486,779,4,2,6.546980,26.314280,-0.862299,1.506848,0.592773,1.134711,0.424306,1.574333e+09,109
1,-11.134070,15.370831,-0.969107,0.393725,0.568159,0.052731,798,4,2,-11.148409,15.363879,-0.941920,1.177285,0.561967,1.334849,0.631927,1.574333e+09,109
2,-9.613478,18.832103,-1.370161,-0.963250,-1.160499,-0.050569,799,4,2,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,109
3,-9.071459,19.484619,-1.359802,1.584012,1.520616,-0.038735,801,1,2,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,109
4,3.386109,28.154275,-1.145692,0.035855,-1.227388,-0.713803,802,0,1,3.373729,28.175323,-1.134802,5.493111,1.930805,2.056836,1.514860,1.574333e+09,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9823,14.436966,-29.002064,-1.011958,1.545032,-7.858073,-0.065161,803,18,2,14.492973,-28.899463,-1.058401,5.089359,2.800400,1.933282,-1.264048,1.574333e+09,143
9824,11.173455,-5.760234,-0.863561,-3.246541,-4.079700,-0.019370,806,18,2,11.161790,-5.686000,-0.918956,7.892776,3.048782,1.537738,0.864142,1.574333e+09,143
9825,15.856948,-36.512585,-1.213429,1.035487,-3.514035,0.136851,802,34,2,14.617583,-29.436074,-1.085488,5.046473,2.802799,1.898150,-1.309933,1.574333e+09,143
9826,14.576085,-29.577559,-1.024024,1.575933,-7.752716,-0.069387,803,18,2,14.617583,-29.436074,-1.085488,5.046473,2.802799,1.898150,-1.309933,1.574333e+09,143


Altre prove, ignorare

In [82]:
# hdf5data = h5py.File('../../Data/data_130.h5', 'r')

In [87]:
# timestamps = h5data['Timestamp']
# df = pd.DataFrame()

# for c, t in enumerate(timestamps):
#     print(c)
#     d = h5data['Sequence'][str(c)]
#     track = np.asarray(d['Tracks'])
#     detection = np.asarray(d['Detections'])
#     row = link_detections_tracks(detection, track)
#     append_df = link_detections_tracks(detection, track)
#     append_df['timestamp'] = t
#     df = df.append(append_df)

0
1
2
3
4
5
6
7
8


ValueError: XA must be a 2-dimensional array.

In [91]:
track.shape[0]

0

In [80]:
np.array(hdf5data['Sequence']['0']['Tracks'])

array([(779, (  6.54805055, 26.36731614, -0.8477613 ), ( 0.20726114,  0.35981265, -0.04548636), 4, 2),
       (798, (-11.13406957, 15.3708309 , -0.96910705), ( 0.39372463,  0.56815921,  0.05273093), 4, 2),
       (799, ( -9.61347787, 18.83210327, -1.37016058), (-0.96325019, -1.16049863, -0.05056859), 4, 2),
       (801, ( -9.07145892, 19.48461852, -1.35980242), ( 1.58401159,  1.52061556, -0.03873475), 1, 2),
       (802, (  3.38610916, 28.15427521, -1.14569238), ( 0.03585469, -1.22738848, -0.71380273), 0, 1)],
      dtype=[('TrackId', '<i8'), ('Pos', [('X', '<f8'), ('Y', '<f8'), ('Z', '<f8')]), ('Vel', [('X', '<f8'), ('Y', '<f8'), ('Z', '<f8')]), ('ClassId', '<u4'), ('Status', '<i4')])

In [78]:
df.loc[0]

X_track             6.54805
Y_track             26.3673
Z_track           -0.847761
vx_track           0.207261
vy_track           0.359813
vz_track         -0.0454864
track_id                779
obj_type                  4
track_status              2
X_box               6.54698
Y_box               26.3143
Z_box             -0.862299
length_box          1.50685
width_box          0.592773
height_box          1.13471
angle_box          0.424306
timestamp       1.57433e+09
camera                  109
Name: 0, dtype: object

In [56]:
# c=0
# d = hdf5data['Sequence'][str(c)]
# track = np.asarray(d['Tracks'])
# detection = np.asarray(d['Detections'])
# row = link_detections_tracks(detection, track)
# append_df = link_detections_tracks(detection, track)
# append_df['timestamp'] = 2

In [85]:
# c=0
# d = hdf5data['Sequence'][str(c)]
# track = np.asarray(d['Tracks'])
# detection = np.asarray(d['Detections'])

# coord_detections = [np.asarray(vals[0].tolist()) for vals in detection]
# coord_tracks = [np.asarray(vals[1].tolist()) for vals in track]
# dists = cdist(coord_tracks, coord_detections, 'euclidean')
# print(type(dists))

<class 'numpy.ndarray'>
